### Variational Autoencoders for sequences hidden conditions estimation

In [14]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# folder = "env"
# sourcedir = currentdir.split(folder)[0]
sys.path.insert(0, parentdir)
# print(parentdir)

import pyro
import pyro.distributions as dist

import vae_seq as vs


1) convert secuences from `.fasta` to torch tensor, batching and padding: 

In [2]:
print("EMBEDDING_DIM: ", vs.EMBEDDING_DIM)
print("BATCH_SIZE: ", vs.BATH_SIZE)

data = vs.get_sequences()
x = data[0]
img_size = seq_size = data.shape[-1]
print("seq_size: ", img_size)

EMBEDDING_DIM:  3
BATCH_SIZE:  10
seq_size:  1143


2) Apply encoder:

In [4]:
encoder = vs.Encoder(x.shape[-1], 3, 10)
# encoder = Encoder(data.shape[-1], 3, 10)
z_loc, z_scale = encoder.forward(x)
print("z_loc, z_scale:")

# batch/z_dim:
print(z_loc.shape)
print(z_scale.shape)


z_loc, z_scale:
torch.Size([10, 3])
torch.Size([10, 3])


3) Apply decoder:

In [9]:
decoder = vs.vae.Decoder(x.shape[-1], 3, 10)
z = pyro.sample("latent", dist.Normal(z_loc, z_scale).to_event(1))
loc_img = decoder.forward(z)
print("loc_img:")
print(loc_img.shape)
    
obs = pyro.sample("obs", dist.Bernoulli(loc_img).to_event(1),
                  obs=x.reshape(-1, img_size))
print("obs (same as x because of sampling restriction):")
print(obs.shape)


hidden.shape:
torch.Size([10, 10])
loc_img.shape:
torch.Size([10, 1143])
loc_img:
torch.Size([10, 1143])
obs (same as x because of sampling restriction):
torch.Size([10, 1143])


/home/valdecar/anaconda3/envs/math/lib/python3.6/site-packages/pyro/primitives.py:86: RuntimeWarning: trying to observe a value outside of inference at obs
  RuntimeWarning)


3) Mix all together in vae (+sampling and conditioning):

In [10]:
ovae = vs.VAE(img_size=img_size, z_dim=3, hidden_dim=10)
print("vae.model(x):")
# x is batch
ovae.model(x)
print("done")
print("vae.guide(x)")
ovae.guide(x)
print("done")


vae.model(x):
x.shape:
torch.Size([10, 1143])
z_loc.shape:
torch.Size([10, 3])
z_scale.shape:
torch.Size([10, 3])
model: z loc_dist.batch_shape:
torch.Size([10])
model: z loc_dist.event_shape:
torch.Size([3])
hidden.shape:
torch.Size([10, 10])
loc_img.shape:
torch.Size([10, 1143])
loc_img.shape:
torch.Size([10, 1143])
model: obs loc_dist.batch_shape:
torch.Size([10])
model: obs loc_dist.event_shape:
torch.Size([1143])
done
vae.guide(x)
guide: latent loc_dist.batch_shape:
torch.Size([10])
guide: latent loc_dist.event_shape:
torch.Size([3])
done
